**Imports**

In [1]:
# Imports
import pandas as pd
import random
import numpy as np
import treelib

**Parameters**

In [2]:
MAX_TREE_DEPTH = 5
MIN_TREE_DEPTH = 2
POP_SIZE = 200
POP_GEN_METHOD = 'grow'

random.seed(10)
randomstate_seed = 10

# Initialise generation method variables
grow = False
if POP_GEN_METHOD is 'grow':
    grow = True

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\david\AppData\Local\Temp\ipykernel_3604\1671493786.py:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if POP_GEN_METHOD is 'grow':


**Dataset imports**

In [3]:
train_df = pd.read_csv(r'training.csv')
test_df = pd.read_csv(r'testing.csv')

In [4]:
# Printing the column headings to better understand the features
print(train_df.columns.values)

['Duration' 'Distance' 'Pickup_longitude' 'Pickup_latitude'
 'Dropoff_latitude' 'Dropoff_latitude.1' 'Haversine' 'Pmonth' 'Pickup_day'
 'Pickup_hour' 'Pickup_minute' 'Pickup_weekday' 'Dropoff_month'
 'Dropoff_day' 'Dropoff_hour' 'Dropoff_minute' 'Dropoff_weekday' 'Temp'
 'Precip' 'Wind' 'Humid' 'Solar' 'Snow' 'GroundTemp' 'Dust']


In [5]:
# Seperate values from data
# Remember: features = data used to reach output, while labels are the outputs themselves
train_labels = train_df[['Duration']]
test_labels = test_df[['Duration']]

train_features = train_df.drop('Duration',axis=1)
test_features = test_df.drop('Duration',axis=1)

print('Labels column: ' + str(train_labels.columns) + '\n\n')
print('Features columns: ' + str(train_features.columns))

Labels column: Index(['Duration'], dtype='object')


Features columns: Index(['Distance', 'Pickup_longitude', 'Pickup_latitude', 'Dropoff_latitude',
       'Dropoff_latitude.1', 'Haversine', 'Pmonth', 'Pickup_day',
       'Pickup_hour', 'Pickup_minute', 'Pickup_weekday', 'Dropoff_month',
       'Dropoff_day', 'Dropoff_hour', 'Dropoff_minute', 'Dropoff_weekday',
       'Temp', 'Precip', 'Wind', 'Humid', 'Solar', 'Snow', 'GroundTemp',
       'Dust'],
      dtype='object')


In [6]:
print(train_features.info())
print('Entries in "training features": ' + str(len(train_features)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720797 entries, 0 to 6720796
Data columns (total 24 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Distance            int64  
 1   Pickup_longitude    float64
 2   Pickup_latitude     float64
 3   Dropoff_latitude    float64
 4   Dropoff_latitude.1  float64
 5   Haversine           float64
 6   Pmonth              int64  
 7   Pickup_day          int64  
 8   Pickup_hour         int64  
 9   Pickup_minute       int64  
 10  Pickup_weekday      int64  
 11  Dropoff_month       int64  
 12  Dropoff_day         int64  
 13  Dropoff_hour        int64  
 14  Dropoff_minute      int64  
 15  Dropoff_weekday     int64  
 16  Temp                float64
 17  Precip              float64
 18  Wind                float64
 19  Humid               float64
 20  Solar               float64
 21  Snow                float64
 22  GroundTemp          float64
 23  Dust                float64
dtypes: float64(13), int64(11

In [7]:
print(train_labels.info())
print('Entries in "training labels": ' + str(len(train_labels)))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6720797 entries, 0 to 6720796
Data columns (total 1 columns):
 #   Column    Dtype
---  ------    -----
 0   Duration  int64
dtypes: int64(1)
memory usage: 51.3 MB
None
Entries in "training labels": 6720797


In [8]:
# Free up RAM
import gc

train_df = None
test_df = None

gc.collect()

0

**Terminal and Function set creation**

In [9]:
# Some basic functions for the function set
def add(x,y): return x + y
def sub(x,y): return x - y
def mul(x,y): return x * y
def div(x,y): return x / y

function_set = [add, sub, mul, div]

In [10]:
# Creation of terminal set
# Terminals are a range of constants, as made below
terminal_set = ['Duration', 'Distance', 'Pickup_longitude', 'Pickup_latitude',
    'Dropoff_latitude', 'Dropoff_latitude.1', 'Haversine', 'Pmonth', 'Pickup_day',
    'Pickup_hour', 'Pickup_minute', 'Pickup_weekday', 'Dropoff_month',
    'Dropoff_day', 'Dropoff_hour', 'Dropoff_minute', 'Dropoff_weekday', 'Temp',
    'Precip', 'Wind', 'Humid', 'Solar', 'Snow', 'GroundTemp', 'Dust']

**Tree definitions**

In [13]:
class GPTree:
    def __init__(self, data = None, left = None, right = None):
        self.data = data
        self.left = left
        self.right = right

    # Allows for either grow (if grow parameter is true) or full method (grow set to false)
    # Makes self into a new random tree
    def random_tree(self, grow, max_depth, depth = 0):
        if (depth < max_depth and not grow) or (depth < MIN_TREE_DEPTH): 
            self.data = function_set[random.randint(0, len(function_set) - 1)]
        elif depth >= max_depth:   
            self.data = terminal_set[random.randint(0, len(terminal_set) - 1)]
        else:
            if random.random() > 0.5: # 50% chance
                self.data = terminal_set[random.randint(0, len(terminal_set) - 1)]
            else:
                self.data = function_set[random.randint(0, len(function_set) - 1)]
        if self.data in function_set:
            self.left = GPTree()
            self.left.random_tree(grow, max_depth, depth = depth + 1)            
            self.right = GPTree()
            self.right.random_tree(grow, max_depth, depth = depth + 1)

    # recursive depth-first evaluation, returns calculated value of tree - can be used on root or any subtree
    def evaluate_tree(self):
        if self.data in terminal_set:
            return self.data
        else:
            left_value = self.left.evaluate_tree()
            right_value = self.right.evaluate_tree()
            own_value = self.data(left_value, right_value)
            return own_value
        
    def create_function(self):
        

    def print_tree(self):
        if self.data in terminal_set:
            print('TERMINAL: ' + str(self.data))
        elif self.data in function_set:
            print('FUNCTION: ' + str(self.data))
            print(self.left.print_tree())
            print(self.right.print_tree())

In [18]:
# Test for random_tree
test_tree = GPTree()
test_tree.random_tree(True, MAX_TREE_DEPTH)
test_tree
test_tree.print_tree()

FUNCTION: <function div at 0x000001A43AE3A9D0>
FUNCTION: <function mul at 0x000001A43AE1CF70>
FUNCTION: <function mul at 0x000001A43AE1CF70>
FUNCTION: <function sub at 0x000001A437EE3550>
FUNCTION: <function mul at 0x000001A43AE1CF70>
TERMINAL: Pickup_day
None
TERMINAL: Dropoff_latitude.1
None
None
TERMINAL: Pmonth
None
None
FUNCTION: <function sub at 0x000001A437EE3550>
FUNCTION: <function div at 0x000001A43AE3A9D0>
TERMINAL: Pickup_weekday
None
TERMINAL: GroundTemp
None
None
FUNCTION: <function div at 0x000001A43AE3A9D0>
TERMINAL: Pmonth
None
TERMINAL: Dropoff_minute
None
None
None
None
FUNCTION: <function div at 0x000001A43AE3A9D0>
FUNCTION: <function sub at 0x000001A437EE3550>
FUNCTION: <function add at 0x000001A437EE3820>
TERMINAL: Pickup_weekday
None
TERMINAL: Dropoff_latitude
None
None
TERMINAL: Pickup_hour
None
None
FUNCTION: <function div at 0x000001A43AE3A9D0>
TERMINAL: Solar
None
FUNCTION: <function mul at 0x000001A43AE1CF70>
TERMINAL: Temp
None
TERMINAL: Pickup_minute
None


**Initial population generation**

In [16]:
# Returns array containing initial tree population
# TODO - look for library that does the tree things but prettier
def init_population(): # Assuming not using ramped 50/50 - TODO, adjust later to allow this
    population = []
    for i in range(0, POP_SIZE):
        new_random_tree = GPTree()
        new_random_tree.random_tree(grow, MAX_TREE_DEPTH)
        population.append(new_random_tree)
    return population

In [17]:
# Test for init_population
test_pop = init_population()
print(len(test_pop))

200


**Fitness function**

In [ ]:
def raw_fitness_function(tree : GPTree, test_dataset):
    tree_output = tree.evaluate_tree()

    # have to convert tree terminals into value from the dataset
    # method - just get whole row of dataframe, and then get the correct terminal from that
    trip_sample = train_df.sample(100, ignore_index=True, random_state=10)
    
     # break into iteratable arrays
    durations = trip_sample.loc[:,"Duration"]
    distances = trip_sample.loc[:,"Distance"]
    pickup_longs = trip_sample.loc[:,"Pickup_longitude"]
    pickup_lats = trip_sample.loc[:,"Pickup_latitude"]
    dropoff_longs = trip_sample.loc[:,"Dropoff_latitude.1"]
    dropoff_lats = trip_sample.loc[:,"Dropoff_latitude"]
    haversines = trip_sample.loc[:,"Haversine"]
    pmonths = trip_sample.loc[:,"Pmonth"]
    pdays = trip_sample.loc[:,"Pickup_day"]
    phours = trip_sample.loc[:,"Pickup_hour"]
    pmins = trip_sample.loc[:,"Pickup_minute"]
    pweekdays = trip_sample.loc[:,"Pickup_weekday"]
    ddays = trip_sample.loc[:,"Dropoff_day"]
    dhours = trip_sample.loc[:,"Dropoff_hour"]
    dmins = trip_sample.loc[:,"Dropoff_minute"]
    dweekdays = trip_sample.loc[:,"Dropoff_weekday"]
    temps = trip_sample.loc[:,"Temp"]
    precips = trip_sample.loc[:,"Precip"]
    winds = trip_sample.loc[:,"Wind"]
    humids = trip_sample.loc[:,"Humid"]
    solars = trip_sample.loc[:,"Solar"]
    snows = trip_sample.loc[:,"Snow"]
    groundtemps = trip_sample.loc[:,"GroundTemp"]
    dusts = trip_sample.loc[:,"Dust"]
    print("Compiling function...")
    func = toolbox.compile(expr=individual)
    print("function compiled")
    fitness = 0
    for i in range(len(trip_sample)):
        # print("fitness eval loop run " + str(i))
        # PSUEDOCODE: individual_fitness = abs(correct_duration - calculated_duration)
        # PSEUDOCODE: fitness = fitness + individual_fitness
        sample_row = trip_sample.iloc[[i]]
        # print("sample row: " + str(type(sample_row)))
        sample_features = sample_row.drop("Duration",axis=1)
        # print("sample features: " + str(type(sample_features)))

        correct_duration = durations[i]
        # print("correct duration for item " + str(i) + " is " + str(correct_duration))
        calculated_duration = func(
            distances[i], pickup_longs[i], pickup_lats[i], dropoff_longs[i], dropoff_lats[i],
            haversines[i], pmonths[i], pdays[i], phours[i], pmins[i], pweekdays[i], ddays[i],
            dhours[i], dmins[i], dweekdays[i], temps[i], precips[i], winds[i], humids[i], solars[i],
            snows[i], groundtemps[i], dusts[i]
        )
        # print("calculated duration for item " + str(i) + " is " + str(calculated_duration))

        individual_fitness = abs(correct_duration - calculated_duration)
        fitness = fitness + individual_fitness
    # print("Fitness: " + str(fitness))
    return fitness,
    
    pass